In [20]:
import seaborn as sns
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


# 1. 누락 데이터 처리

In [22]:
nan_deck = df['deck'].value_counts(dropna = False) # dropna = False: NaN 값 구할 수 있다.
                                                    # dropna = True : NaN 값을 제외한 유효한 데이터 개수를 구한다.
print(nan_deck)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


In [23]:
# 누락 데이터 찾기
df.head().isnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [24]:
df.head().notnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


In [25]:
df.head().isnull().sum(axis=0) # 행들을 더하기

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64

In [26]:
df.head().isnull().sum(axis=1) # 열들을 더하기

0    1
1    0
2    1
3    0
4    1
dtype: int64

In [33]:
missing_df = df.isnull()
for col in missing_df.columns:
    missing_count = missing_df[col].value_counts() # 각 열의 NaN 개수 파악
    
    try:
        print(col, ':', missing_count[True]) # NaN 값이 있으면 개수 출력
    except:
        print(col,':',0) # NaN 값이 없으면 0개 출력
        
# try & except: try 하위 명령에서 오류가 발생할 경우, except 실행

survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0


In [9]:
# 누락 데이터 제거
# NaN 값이 500개 이상인 모든 열 삭제
df_thresh = df.dropna(axis = 1, thresh=500) # NaN 값을 500개 이상 갖는 경우 삭제
                                            # 임계값(threshold)을 설정한다고 생각하면 된다.
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


In [10]:
# age 열에 나이 데이터가 없는 모든 행 삭제
df_age = df.dropna(axis = 0, subset = ['age'], how = 'any') # subset : 열을 list 형태로 입력
                                                            # how = any’: NaN값이 하나라도 존재하면 삭제
                                                            # how = 'all’: 모든 데이터가 NaN값일 때 삭제

print(len(df_age))

714


In [11]:
# NaN 값을 평균값으로 치환하기
print(df['age'].head(10))
print('\n')

mean_age = df['age'].mean()
df['age'].fillna(mean_age, inplace = True)

print(df['age'].head(10))

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64


0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


In [12]:
# DataFrame 만들고 NaN 값 치환하기
import numpy as np
import pandas as pd

df_ex = pd.DataFrame([[np.nan,1,3],
                     [5,np.nan,4],
                     [8,6,np.nan]],
                    columns = ('A','B','C'))
print(df_ex)

     A    B    C
0  NaN  1.0  3.0
1  5.0  NaN  4.0
2  8.0  6.0  NaN


In [13]:
median_ex = df_ex['A'].median()
df_ex['A'].fillna(median_ex, inplace = True)

# B열의 NaN 값을 0으로, C 열의 NaN 값을 9로 채우기
values = {'B':0, 'C':9}
df_ex.fillna(value = values, inplace = True)
df_ex

,A,B,C
0,6.5,1.0,3.0
1,5.0,0.0,4.0
2,8.0,6.0,9.0


In [14]:
print(df['embark_town'][825:830])
print('\n')

most_freq = df['embark_town'].value_counts(dropna = True).idxmax() # value_counts 메서드를 사용하여 승선도시별 승객 수를 찾고
                                                                    # idxmax() 메서드로 가장 큰 값을 갖는 도시를 찾는다
print(most_freq)
print('\n')

df['embark_town'].fillna(most_freq, inplace = True)

print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


Southampton


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object


In [15]:
# 이웃하고 있는 값으로 바꾸기
import seaborn as sns
df = sns.load_dataset('titanic')

print(df['embark_town'][825:830])
print('\n')

# embark_town 열의 NaN 값을 바로 앞에 있는 828행의 값으로 변경하기
df['embark_town'].fillna(method = 'ffill', inplace = True)
print(df['embark_town'][825:830])

# method = ‘bfill’은 NaN 행의 다음 행 값으로 바꾼다.

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object


# 2. 중복 데이터 처리

In [16]:
import pandas as pd
df = pd.DataFrame({'c1':['a','a','b','a','b'],
                  'c2':[1,1,1,2,2],
                  'c3':[1,1,2,2,2]})
print(df)
print('\n')

# 중복 데이터 확인
df_dup = df.duplicated() # 전에 나온 행들과 비교하여 중복되는 행이면 True를 반환
                         # 처음 나오는 행에 대해서는 False 반환
print(df_dup)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


0    False
1     True
2    False
3    False
4    False
dtype: bool


In [17]:
col_dup = df['c2'].duplicated()
print(col_dup)

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


In [18]:
# 중복 데이터 제거
print(df)
print('\n')

df2 = df.drop_duplicates() # 중복되는 행을 제거하고 고유한 관측값을 가진 행들만 남긴다.
print(df2)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [19]:
df3 = df.drop_duplicates(subset = ['c2','c3']) # subset 옵션에 '열 이름 리스트' 전달
                                                # 'c2','c3' 열을 기준으로 판별
print(df3)

  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
